In [338]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno
import itertools
import os

In [339]:
file = "data/2021/raw-chicago-daily-max-temp.csv"
output = "data/2021/chicago-daily-max-temp.csv"

In [340]:
df = pd.read_csv(file, header=None, delimiter=";", index_col=0)
df.drop(columns=[n for n in range(1,25) if n % 2 == 0], inplace=True)

In [341]:
print(len(df.columns))

MONTHS = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df.columns = MONTHS

12


In [342]:
df.head()

,January,February,March,April,May,June,July,August,September,October,November,December
0,,,,,,,,,,,,
1,35,33,35,40,85,77,76,76,76,85,49,49
2,35,33,43,51,85,78,73,78,78,80,47,59
3,33,36,59,71,73,87,86,83,75,76,45,51
4,30,39,37,76,58,91,89,85,73,73,47,44
5,37,19,49,76,61,92,91,87,82,67,52,41


In [343]:
for col in df.columns:
    print(col, df[col].unique())

January [35 33 30 37 39 38 36 29 41 40 34 32 27 26]
February ['33' '36' '39' '19' '12' '5' '15' '18' '16' '21' '17' '4' '20' '27' '26'
 '29' '37' '40' '47' '45' '43' '52' '49' 'M' '-']
March [35 43 59 37 49 48 52 62 69 64 54 40 38 39 45 58 65 68 50 66 51 44]
April ['40' '51' '71' '76' '81' '82' '69' '62' '55' '56' '54' '57' '52' '43'
 '50' '60' '64' '47' '79' '87' '-']
May [85 73 58 61 56 60 54 57 52 62 70 71 72 76 75 86 88 82 90 59 49 69]
June ['77' '78' '87' '91' '92' '89' '80' '90' '93' '83' '88' '82' '72' '73'
 '79' '76' '84' '-']
July [76 73 86 89 91 68 71 74 80 84 79 77 82 85 88 75 92 90 94 87]
August [76 78 83 85 87 89 92 93 88 84 80 86 91 95 94 90 82]
September ['76' '78' '75' '73' '82' '88' '79' '77' '80' '87' '86' '89' '63' '67'
 '69' '83' '-']
October [85 80 76 73 67 68 75 79 77 65 71 62 59 66 72 56 51 55 57 53]
November ['49' '47' '45' '52' '59' '65' '58' '56' '57' '43' '39' '37' '36' '61'
 '40' '50' '55' '53' '48' '31' '42' '41' '-']
December [49 59 51 44 41 38 26 35 46 60

In [344]:
df.replace(['M'], np.NaN, inplace=True)

In [345]:
def fahrenheit_to_celcius(f):
    if f == "-":
        return f
    return (float(f) - 32) * 5./9

df = df.applymap(fahrenheit_to_celcius)

In [346]:
prod = itertools.product(MONTHS, df.index)
# new_index = [" ".join([str(col), str(ind)]) for col, ind in prod if df.loc[ind, col] != "-"]
new_index = [" ".join([str(col), str(ind)]) for col, ind in prod]

In [347]:
# pd.wide_to_long(df=df, stubnames=['2021'], i=df.columns, j="temperature")
df = df.transpose().stack()
df = pd.DataFrame(df)
df.head()

0
        0          
January 1  1.666667
        2  1.666667
        3  0.555556
        4 -1.111111
        5  2.777778

In [348]:
indexes = list(df.index.values)
curr_index = [" ".join([str(col), str(ind)]) for col, ind in indexes]
print(curr_index.__len__(), new_index.__len__())

new_index.remove('February 29')

compare = pd.DataFrame({'current': curr_index, 'new': new_index})

371 372


In [349]:
df.columns = df.columns.to_flat_index()
df.index = new_index
df.columns = ['max_temperature']
df.drop(index=df.loc[df['max_temperature']=="-"].index, inplace=True)
df.head()

,max_temperature
January 1,1.666667
January 2,1.666667
January 3,0.555556
January 4,-1.111111
January 5,2.777778


In [350]:
df.index = [date + ", 2021" for date in df.index.values]
df.index.rename(name='date', inplace=True)

In [351]:
df.to_csv(output)